# Example of Linear Regression using GapMinder Data

As a very simple example of machine learning, we're going see if we can predict life expectancy using some data from [GapMinder](https://www.gapminder.org/tools/#_data_/_lastModified:1526038652718&lastModified:1526038652718;&chart-type=bubbles), an organisation that aims to educate us more on the true state of the world.
The input data set will have 136 countries with the following variables:
- Continent
- Life expectancy
- GDP per capita, PPP, inflation adjusted
- Healthcare spend as a percentage of GDP
- Population per square km 
- The democracy index of the country (high is better). See https://en.wikipedia.org/wiki/Democracy_Index

Go to [GapMinder](https://www.gapminder.org/tools/#$state$time$value=2007;;&data$_lastModified:1526038652718&lastModified:1526038652718;&chart-type=bubbles) now and take a look at some of these variable in the data viewer.  
- Do any of the variables (visually) look like they have a bearing on life expectancy?  
- Are any of them surprising?

#### Question: is there a country that is particularly inefficient in terms of healthcare spend?
<img src="https://github.com/DataForGood-Norway/GirlsCanDoIt/blob/master/MachineLearning/Lab/files/Screen%20Shot%20from%20gapminder.png?raw=true" alt="GDP per Capita vs. Life Expectancy from Gapminder" title="GDP per Capita vs. Life Expectancy from Gapminder" style="width: 50pc;"/>

# How do we know which variables to use?
When we have a number of variables, it may not be immediately apparent which ones influence the final result.  We will usually find that we can and should  drop one or more independent variables because:
- We don't want to include variables that aren't significant
<img src="https://github.com/DataForGood-Norway/GirlsCanDoIt/blob/master/MachineLearning/Lab/files/GarbageInGarbageOut.png?raw=true" alt="Garbage In - Garbage out" title="Garbage In - Garbage Out" style="width: 25pc;"/>
- When you are presenting your results you will have to explain why you included a variable; "because I had the data" is not a good enough reason!

There are [several ways to acheive this](https://www.analyticsvidhya.com/blog/2016/12/introduction-to-feature-selection-methods-with-an-example-or-how-to-select-the-right-variables/) but we will concentrate on "backward elimination".  To understand this you need to know about the "P" value.  A proper explanation is [here](https://www.mathbootcamps.com/what-is-a-p-value) but for now we'll just say that the lower the P value is, the more significant the variable.

## Backward elimination
1. Choose a maximum P value: 5% is a good value
2. Run the linear regression using all the dependent variables. 
3. Look at the P values from the output, and choose the biggest.  
4. If it is > than 5%, then drop that variable.
5. Rerun the linear regression.
6. Repeat steps 2-5 until all P-values are < 5%.  

#### I will take you up to step 5, then the rest will be up to you!

# Coding
Firstly we need to import some libraries

In [ ]:
# Standard library for numerical analysis
import numpy as np
# Standard library for data manipulation
import pandas as pd

# User-defined library for making plots
import sys
!mkdir -p local_modules/
!cd local_modules
!wget https://raw.githubusercontent.com/DataForGood-Norway/GirlsCanDoIt/master/MachineLearning/Lab/python/plot_functions.py
!cd ..
sys.path.append('local_modules')
from plot_functions import make_plot

## Importing the Dataset

Next we will import the dataset with data for 2007, which has the following columns:
- Country - 136 countries are included
- Continent
- lifeexp - Life expectancy
- gdpPercap - GDP per capita, PPP, inflation adjusted
- health_spend - Healthcare spend as a percentage of GDP
- Pop density - popultaion per square km
- Democracy - The democracy index of the country

In [ ]:
dataset = pd.read_csv('https://raw.githubusercontent.com/DataForGood-Norway/GirlsCanDoIt/master/MachineLearning/Lab/datasets/gapminder_2007_emma.csv')
dataset.head(13)

### Data preparation
Firstly the data now needs to be split into independent and dependent variables:

In [ ]:
# Country, continent, population, GDP per cap, healthcare spend
# pop density, democracy
# So we keep all lines (':') but get rid off the 'lifeExp' column
X_df = dataset.loc[:, dataset.columns != 'lifeExp']
# Will use the log of the GDP per capita because from our first look
# there seems to be a log-linear realtionship
# The log function is applied to every element x of the column 'gdpPercap'
X_df['logGDP'] = X_df['gdpPercap'].apply(lambda x: np.log(x))
# Then we can get rid of 'gdpPercap'
X_df = X_df.drop('gdpPercap', 1)

# The dependent variable is life expectancy, labelled 'lifeExp'
y_df = dataset['lifeExp']
# Also making a list of continents for plotting purposes
cont = dataset.loc[:,'continent'].tolist()

X_df.head(13)

## Cleaning
To get ready for the regression we need to make sure that all the columns contain numbers only:
- Cells that are empty currently contain "Not a Number" (nan) - these will be replaced with an average
- Cells that contain words ("categorical variables") will be replaced with numerical values. 

In [ ]:
# Fix the "NaNs" - replace with the average of that column
# sklearn contains libraries for preprocessing data
# now importing Imputer class
from sklearn.preprocessing import Imputer
# Create object
imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
# Fit imputer object to feature X to every column with numerical values
imputer = imputer.fit(X_df.loc[:,['pop', 'health_spend', 'Pop density', 'Democracy', 'logGDP']])
# Replace missing data with replaced values
X_df.loc[:,['pop', 'health_spend', 'Pop density', 'Democracy', 'logGDP']] = imputer.transform(X_df.loc[:,['pop', 'health_spend', 'Pop density', 'Democracy', 'logGDP']])
X_df.head(13)

## Categorical variables
Categorical variables are ones which have a name, not a number.  In this case that would "Country name" and "Continent".

### Country name
We only have each country listed once, and it makes no sense to use the name of the country in the regression, so we are simply going to number them 0-136

In [ ]:
# now importing LabelEncoder class
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Encoding the Independent Variable
# Firstly assign the categorical variables a unique number using LabelEncoder
labelencoder_X = LabelEncoder()
# Country names are in column zero
# Each country will get a number 0-136 (for the full dataset) 
X_df[['country']] = labelencoder_X.fit_transform(X_df.loc[:,'country'])
X_df.head(13)

### Continent
There are 5 continents in this dataset: Africa, Americas, Asia, Europe and Oceania.  If we simply give them a number 0-4, Python will assume that Oceania "is greater" than all this other continents, which makes no sense.  To fix this we use "one-hot encoding" which works like this:
- Number the continents 0-4
- Create a column for each continent -- these new columns are called __dummy variables__
- If the country is in that continent the column contains "1" otherwise it contains "0"

In [ ]:
# now importing OneHotEncoder class
from sklearn.preprocessing import OneHotEncoder
cont = labelencoder_X.fit_transform(X_df.loc[:,'continent'])
# one hot encode continents

# use pd.concat to join the new columns with your original dataframe
X_df = pd.concat([X_df,pd.get_dummies(X_df['continent'], prefix='continent')], axis=1)

# now drop the original 'continentG' column (you don't need it anymore)
X_df.drop(['continent'],axis=1, inplace=True)
X_df.head(13)

## Dummy variable trap
A requirement of regression is that no column can be predicted from the others. Since any of the five columns could be predicted from the other four (known as the __dummy variable trap__) we need to drop one.
- Normally it would be the first column, but since Africa looks quite interesting in this dataset, I decided to drop Oceania

In [ ]:
X_df.drop(['continent_Oceania'], axis=1, inplace=True)
X_df.head(13)

## Add a constant
We need to add a column of ones at the beginning, as the regression requires a constant

In [ ]:
#X = np.append(arr = np.ones((len(X),1)).astype(int), values = X, axis = 1)
X_df['constant_for_regression'] = 1
X_df.head()


## Splitting into tesing and training set
This is very important!  We will first try to create the model from the training set and then apply it to the test set to see if the model has worked.

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, cont_train, cont_test = train_test_split(X_df, y_df, cont, test_size = 0.2, random_state = 0)
X_train.head()

## Training the regressor
This is where we get to the interesting part!

Firstly we will train the regressor using all the variables and see how it performs:


In [ ]:
"""Start the training
"""

#train the regressor
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train,y_train)
#Make predictions
y_pred = regressor.predict(X_train)

#plot the results
make_plot('GDP per cap',np.exp(X_train['logGDP']),y_train,y_pred,cont_train)
#plot the results
make_plot('Health spend',X_train['health_spend'],y_train,y_pred,cont_train)


#### Question: What can we say about the above plots?  
#### Question: Is there a continent that stands out from the rest?

## Start backward elimination
As a reminder:
1. Choose a maximum P value: 5% is a good value
2. Run the linear regression using all the dependent variables.
3. Look at the P values from the output, and choose the biggest.
4. If it is > than 5%, then drop that variable.
5. Rerun the linear regression.
6. Repeat steps 2-5 until all P-values are < 5%.

#### The code below is the first step and then you will have a chance to get involved and complete the other steps.


In [ ]:
import statsmodels.formula.api as sm

In [ ]:
#Start backward elimination
X_opt = X_train.loc[:,['constant_for_regression', 'continent_Africa',
                       'continent_Americas','continent_Asia',
                       'continent_Europe','country',
                       'pop','health_spend','logGDP','Pop density',
                       'Democracy']]
# X_opt.head(13)
regressor_OLS = sm.OLS(endog = y_train, 
                      exog = X_opt).fit()
regressor_OLS.summary()

#### Look at the table above.  We are particularly interested in the P-values for each variable.  What does this say about the significance of the different variables?  Are any of them surprising?

#### You can plot the results using the user defined module make_plot, which sets some of the display parameters like colour and titles, without you having to type it all in every time.  The format is:
make_plot('independent_variable',x_data,actual_data,predicted data,continents)

In [ ]:
#Make predictions
regressor_new = LinearRegression()
regressor.fit(X_opt,y_train)
y_pred_new = regressor.predict(X_opt)

#plot the results
gdp_per_cap = np.exp(X_train['logGDP'])
make_plot('GDP per cap',gdp_per_cap,y_train,y_pred_new,cont_train)
#plot the results
make_plot('Health spend',X_train['health_spend'],y_train,y_pred_new,cont_train)

## Now your turn!
#### Use the cells below to drop some variables and make new predictions

In [ ]:
# Backward elimination
X_opt = X_train.loc[:,['constant_for_regression', 
                       'continent_Africa','continent_Americas',
                       'continent_Asia','continent_Europe',
#                       'country',
                       'pop',
                       'health_spend',
                       'logGDP',
                       'Pop density',
                       'Democracy'
                      ]]
# X_opt.head(13)
regressor_OLS = sm.OLS(endog = y_train, 
                      exog = X_opt).fit()
regressor_OLS.summary()

In [ ]:
# Make predictions
regressor_new = LinearRegression()
regressor_new.fit(X_opt,y_train)
y_pred_new = regressor_new.predict(X_opt)

#plot the results
gdp_per_cap = np.exp(X_train['logGDP'])
make_plot('GDP per cap',gdp_per_cap,y_train,y_pred_new,cont_train)
#plot the results
make_plot('Health spend',X_train['health_spend'],y_train,y_pred_new,cont_train)

#### Check the results on the test dataset

In [ ]:
# Select the same features as before
X_opt_test = X_test.loc[:,['constant_for_regression', 
                       'continent_Africa','continent_Americas',
                       'continent_Asia','continent_Europe',
#                       'country',
                       'pop',
                       'health_spend',
                       'logGDP',
                       'Pop density',
                       'Democracy'
                      ]]


# Make predictions
# regressor_test = LinearRegression()
# regressor_test.fit(X_opt_test,y_train)
y_pred_test = regressor_new.predict(X_opt_test)

#plot the results
gdp_per_cap = np.exp(X_test['logGDP'])
make_plot('GDP per cap',gdp_per_cap,y_test,y_pred_test,cont_test)
#plot the results
make_plot('Health spend',X_test['health_spend'],y_test,y_pred_test,cont_test)